In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import json
import sys
sys.path.append("..")
from scripts.process import *
from scripts.predict import *

In [2]:
field = pd.read_csv('../data/new_field.csv')

with open('../data/det.json', 'r') as file:
    det_table = json.load(file)



In [3]:
det = pd.DataFrame(det_table)
det.head()

,species,eye_size,snout_shape,parr_marks,parr_marks_length,spotting_density,fin_type,parr_marks_spacing,spotting_characteristic
0,ck,large,pointy,slightly faded,long,medium,anal fin,wider than interspaces,circle
1,co,large,short and blunt,slightly faded,long,medium,anal fin,narrower than interspaces,circle
2,cm,medium,NA,faded,short,medium,caudal fin,NA,variable
3,pink,medium,NA,NA,NA,NA,caudal fin,half,NA
4,so,very large,NA,slightly faded,irregular,NA,caudal fin,variable,row


In [4]:
field.head()

,tag_id_long,watershed,river,site,method,local,water_temp_start,fork_length_mm,species
0,989.001038884511,nanaimo,nanaimo,jack point,beach seine,marine,10.6,80.0,ck
1,989.001038885629,nanaimo,nanaimo,jack point,beach seine,marine,13.3,NaN,ck
2,989.001038888882,nanaimo,nanaimo,jack point,beach seine,marine,14.1,76.0,ck
3,989.001038889013,nanaimo,nanaimo,jack point,beach seine,marine,10.6,76.0,ck
4,989.001038888642,nanaimo,nanaimo,jack point,beach seine,marine,10.6,85.0,ck


In [5]:
processed_data = processing(data=field, det_data=det_table)

In [6]:
processed_data.head()

,tag_id_long,water_temp_start,fork_length_mm,species,watershed_black creek,watershed_chemainus,watershed_cowichan,watershed_englishman,watershed_koksilah,watershed_nanaimo,...,parr_marks_spacing_NA,parr_marks_spacing_half,parr_marks_spacing_narrower than interspaces,parr_marks_spacing_variable,parr_marks_spacing_wider than interspaces,spotting_characteristic_NA,spotting_characteristic_circle,spotting_characteristic_irregular,spotting_characteristic_row,spotting_characteristic_variable
0,989.001038884511,10.6,80.0,ck,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
1,989.001038885629,13.3,None,ck,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
2,989.001038888882,14.1,76.0,ck,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
3,989.001038889013,10.6,76.0,ck,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
4,989.001038888642,10.6,85.0,ck,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0


In [7]:
det_data,prob_data = preprocess_data(processed_data)

In [8]:
prob_data.head()

,tag_id_long,species,water_temp_start,fork_length_mm,watershed_cowichan,watershed_englishman,watershed_nanaimo,watershed_puntledge,river_center creek,river_cowichan,...,snout_shape_short and rounded,parr_marks_NA,parr_marks_faded,parr_marks_slightly faded,parr_marks_length_long,parr_marks_length_short,spotting_density_high,spotting_density_medium,fin_type_anal fin,fin_type_caudal fin
0,989.001038884511,ck,10.6,80.0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,1,1,0
1,989.001038885629,ck,13.3,None,0,0,1,0,0,0,...,0,0,0,1,1,0,0,1,1,0
2,989.001038888882,ck,14.1,76.0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,1,1,0
3,989.001038889013,ck,10.6,76.0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,1,1,0
4,989.001038888642,ck,10.6,85.0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,1,1,0


In [9]:
det_data.head()

,tag_id_long,species,eye_size_large,eye_size_medium,eye_size_small,eye_size_very large,snout_shape_NA,snout_shape_long and pointy,snout_shape_pointy,snout_shape_short and blunt,...,parr_marks_spacing_NA,parr_marks_spacing_half,parr_marks_spacing_narrower than interspaces,parr_marks_spacing_variable,parr_marks_spacing_wider than interspaces,spotting_characteristic_NA,spotting_characteristic_circle,spotting_characteristic_irregular,spotting_characteristic_row,spotting_characteristic_variable
0,989.001038884511,ck,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
1,989.001038885629,ck,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
2,989.001038888882,ck,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
3,989.001038889013,ck,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
4,989.001038888642,ck,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0


In [10]:
def voting_classifier_deterministic(data):
    random_numbers = [42, 231, 351, 701, 996, 523, 710, 686, 568, 268]

    X = data.drop(['species','tag_id_long'], axis=1)
    y = data['species']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    models = []
    for num in random_numbers:
        max_depth = 8
        dt = DecisionTreeClassifier(max_depth=max_depth, random_state=num)
        dt.fit(X_train, y_train)
        models.append(dt)
        
    all_predictions = []
    for dt in models:
        predictions = dt.predict(X)
        all_predictions.append(predictions)

    final_predictions = []
    for i in range(len(X)):
        row_predictions = [pred[i] for pred in all_predictions]
        prediction = max(set(row_predictions), key=row_predictions.count)
        final_predictions.append(prediction)

    det_data['prediction'] = final_predictions

    return det_data[['tag_id_long','prediction']]

In [11]:
det_results = voting_classifier_deterministic(processed_data)

C:\Users\hp\AppData\Local\Temp\ipykernel_19576\2835707174.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  det_data['prediction'] = final_predictions


In [12]:
def voting_classifier_probabilistic(data): 

    X = data.drop(['species', 'tag_id_long'], axis=1)
    y = data['species']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    dt = DecisionTreeClassifier(max_depth = 7, min_samples_split = 6, min_samples_leaf = 4, 
                                random_state = 42)
    dt.fit(X_train, y_train)

    scaler = StandardScaler()
    X['fork_length_mm'] = scaler.fit_transform(X[['fork_length_mm']])
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
    y_label = tf.keras.utils.to_categorical(y_enc)
    X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

    num_features = X.shape[1]
    dl_model = tf.keras.Sequential([
        layers.Input(shape=(num_features,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(8, activation='softmax')  #change based on number of labels
    ])
    dl_model.compile(optimizer=Adam(learning_rate=0.0001),        
    loss='categorical_crossentropy',  
    metrics=['accuracy']) 
    dl_model.fit(X_train, y_train, 
                    epochs = 20, 
                    batch_size = 32, 
                    validation_split=0.2)


    dt_pred = dt.predict(X)

    dl_pred = dl_model.predict(X) 
    dl_pred_df = pd.DataFrame(dl_pred, columns=le.classes_) # in a table format showing all the confidences for each label
    max_label = [] # to extract the label with highest confidence
    for i in range(dl_pred_df.shape[0]):
        max_label.append(dl_pred_df.iloc[i].idxmax())


    prob_data['dt_prediction'] = dt_pred
    prob_data['dl_prediction'] = max_label

    return prob_data[['tag_id_long','dt_prediction', 'dl_prediction']]



In [13]:
prob_results = voting_classifier_probabilistic(processed_data)

Epoch 1/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4338 - loss: 2.0082 - val_accuracy: 0.4289 - val_loss: 1.6255
Epoch 2/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4445 - loss: 1.4817 - val_accuracy: 0.4289 - val_loss: 1.1521
Epoch 3/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4422 - loss: 1.0895 - val_accuracy: 0.5177 - val_loss: 0.9888
Epoch 4/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5075 - loss: 0.9728 - val_accuracy: 0.5177 - val_loss: 0.9463
Epoch 5/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5135 - loss: 0.9335 - val_accuracy: 0.5177 - val_loss: 0.9332
Epoch 6/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5093 - loss: 0.9309 - val_accuracy: 0.5177 - val_loss: 0.9265
Epoch 7/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5056 - loss: 0.9191 - val_accuracy: 0.5177 - val_loss: 0.9222
Epoch 8/20
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5060 - loss: 0.9223 - 

C:\Users\hp\AppData\Local\Temp\ipykernel_19576\1637021963.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_data['dt_prediction'] = dt_pred
C:\Users\hp\AppData\Local\Temp\ipykernel_19576\1637021963.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prob_data['dl_prediction'] = max_label


In [14]:
prob_results.head()

,tag_id_long,dt_prediction,dl_prediction
0,989.001038884511,ck,ck
1,989.001038885629,ck,ck
2,989.001038888882,ck,ck
3,989.001038889013,ck,ck
4,989.001038888642,ck,ck


In [15]:
def voting_classifier(det_results,prob_results):
    df = det_results.merge(prob_results,on='tag_id_long',how='left')
    df.columns = ['tag_id_long','pred_1','pred_2','pred_3']

    ensemble_pred = []
    for row in range(len(df)):
        prediction = [df.iloc[row]['pred_1'],
                      df.iloc[row]['pred_2'],
                      df.iloc[row]['pred_3']]
        if 'pink' in prediction:
            ensemble_pred.append('pink')
        elif 'so' in prediction:
            ensemble_pred.append('so')
        else:
            ensemble_pred.append(max(set(prediction), key=prediction.count))


    df['prediction'] = ensemble_pred
    
    return df
        
        

In [16]:
df = voting_classifier(det_results,prob_results)

In [17]:
df

,tag_id_long,pred_1,pred_2,pred_3,prediction
0,989.001038884511,ck,ck,ck,ck
1,989.001038884511,ck,ck,ck,ck
2,989.001038885629,ck,ck,ck,ck
3,989.001038888882,ck,ck,ck,ck
4,989.001038889013,ck,ck,ck,ck
...,...,...,...,...,...
178996,NaN,pink,stl,ck,pink
178997,NaN,pink,stl,ck,pink
178998,NaN,pink,stl,ck,pink
178999,NaN,pink,stl,ck,pink
